In [ ]:
import os
import json
from six import string_types
from translating_trees import Node
from translating_trees import pretty_print_tree

In [ ]:
def figure_out_csharp(json_lists):
#     print(len(json_lists))
    all_tokens = set()
    children = {}
    parents = {}
    max_vars = 0
    max_consts = 0
    max_chars = 0
    max_strings = 0
    
    def tally_tokens(prog, parent=None):
        all_vars = set()
        all_consts = set()
        all_chars = set()
        all_strings = set()
        
        
        
        # ugly special cases
        if parent == "ArgumentList" and type(prog) == type("") and prog[0] == "(" and prog[-1] == ")":
            root_val = "()"
          
        if type(prog) == type("") or type(prog) == int: #TODO: is this second case necesary?
            # Add to parents #TODO: How are floats/doubles represented?
            if prog in parents:
                parents[prog].add(parent)
            else:
                parents[prog] = set([parent])
                
            return all_vars, all_consts, all_chars, all_strings
        
        for key in prog.keys():
             
            root_val = key
            
            if not type(root_val) == type(""):
                print("AAAAAAA")

            all_tokens.add(root_val)
            # Add to parents
            if root_val in parents:
                parents[root_val].add(parent)
            else:
                parents[root_val] = set([parent])
            # Add to children
            if parent in children:
                children[parent].add(root_val)
            else:
                children[parent] = set([root_val])
                
            if root_val == "IdentifierName":
                all_vars.add(prog[key])
            elif root_val == "NumericLiteralExpression":
                all_consts.add(prog[key])
            elif root_val == "CharacterLiteralExpression":
                all_chars.add(prog[key])
            elif root_val == "StringLiteralExpression":
                all_strings.add(prog[key])
            elif root_val == "VariableDeclarator" and type(prog[key]) == type(""):
                all_vars.add(prog[key])
            else:
                child_keys = []
                
                if type(prog[key]) == type([]):
                    for child in prog[key]:
                        for child_key in child.keys():
                            child_keys.append((child_key, child))
                elif type(prog[key]) == type(""):
                    child_keys.append((prog[key], prog[key]))
                else:
                    print("something weird is going on", prog)
                
                # Add to children
                for c in child_keys:
                    child_key, child = c
                            
                    child_vars, child_consts, child_chars, child_strings = tally_tokens(child, root_val)
                    all_vars |= child_vars
                    all_consts |= child_consts
                    all_chars |= child_chars
                    all_strings |= child_strings
                    
            return all_vars, all_consts, all_chars, all_strings
    
    i = 0
    for json_list in json_lists:
        print("new list")
        for prog in json_list:
            i += 1
            all_vars, all_consts, all_chars, all_strings = tally_tokens(prog)
            max_vars = max(max_vars, len(all_vars))
            max_consts = max(max_consts, len(all_consts))
            max_chars = max(max_chars, len(all_chars))
            max_strings = max(max_strings, len(all_strings))
#             if len(all_strings) > 1000:
#                 print("UH OH ON", i)
#                 print(all_consts)
#                 print(all_strings)
#                 print("======")
#                 print(prog)
                
#     print(all_tokens)
#     print(children)
#     print(parents)
#     print(max_vars)
#     print(max_consts)
#     print(max_chars)
#     print(max_strings)
    return all_tokens, children, parents, max_vars, max_consts, max_chars, max_strings

In [ ]:
# all_tokens, children, parents, max_vars, max_consts, max_chars, max_strings = figure_out_csharp(json_lists)

In [ ]:
def get_files(folder, suffix):
    matching_files = []
    for path, dirs, files in os.walk(folder):
        for name in files:
            if name[-len(suffix):] == suffix:
                matching_files.append(name)
    return matching_files

In [ ]:
def haskell_voodoo():
    files = get_files("filtered_data", "cs.txt")
    os.system("mkdir jsonified")
    os.system("ghc CleanCSharpData.hs -main-is CleanCSharpData.main -o CleanCSharpData;")
    for file in files:
        os.system("./CleanCSharpData filtered_data/{} > jsonified/{}".format(file, file))

In [ ]:
# Get all the C# datasets
def load_data(folder, extension):
    all_files = get_files(folder, extension)

    i = 0
    total_count = 0
    correct_count = 0

    json_lists = []
    for file in all_files:
        print("processing file", file)
        j_list = []
        with open(folder + "/" + file) as f:
            lines = f.readlines()
            print(len(lines))
            for line in lines:
                total_count += 1
                try:
                    prog = json.loads(line)
                    correct_count += 1
                    if not prog == []:
                        j_list.append(prog)
                except:
                    pass
                i += 1
        json_lists.append(j_list)
            
    print("TOTAL:", total_count)
    print("CORRECT:", correct_count)
    print(correct_count/total_count)
    return json_lists

In [ ]:
# haskell_voodoo()
# all_tokens, children, parents, max_vars, max_consts, max_chars, max_strings = figure_out_csharp(json_lists)

In [ ]:
# Blatantly copied from ProgramMatching.py
def isMatch(jprog, cprog):
    if not (len(jprog) == 3 and len(cprog) == 3):
        return False
    if jprog[2] == "[]" or cprog[2] == "[]":
        return False
    try:
        jName = jprog[1].replace('_','').lower()
        cName = cprog[1].replace('_','').lower()
        jFile = jprog[0][:jprog[0].index('.')].lower()
        if "/" in jFile:
            jFile = jFile.split("/")[-1]
        cFile = cprog[0][:cprog[0].index('.')].lower()
        if "/" in cFile:
            cFile = cFile.split("/")[-1]
        return (jName == cName) and (jFile == cFile)
    except:
        print("======JJJJJJJ======")
        print(jprog)
        print("=======CCCCCCC========")
        print(cprog)
        return "weirdness"

def filter_files(jfile, cfile, orig_folder, save_folder):
    d = {}
    
    jlist = []
    clist = []
    with open(orig_folder + "/" + jfile) as f:
        jlist = [line.split("\t") for line in f.readlines()]
        
    with open(orig_folder + "/" + cfile) as f:
        clist = [line.split("\t") for line in f.readlines()]
        
    i = 0
    for jprog in jlist:
        if not i%1000:
            print("i is ", i)
        i+=1
        for cprog in clist:
            match = isMatch(jprog,cprog)
            if match == "weirdness":
                return
            elif match:
                d[jprog[2]] = cprog[2]
    all_files = [(k,v) for k,v in d.items()]
    print("ok", len(all_files))
    # Write java
    with open(save_folder + "/" + jfile, "w") as f:
        for file in all_files:
            f.write(file[0])
    
    # Write C#
    with open(save_folder + "/" + cfile, "w") as f:
        for file in all_files:
            f.write(file[1])
            
        
    

In [ ]:
def filter():
    csharp_files = sorted(get_files("j2c-raw-data", "cs.txt"))
    java_files = sorted(get_files("j2c-raw-data", "java.txt"))[3:]

    print(csharp_files)
    print(java_files)

    for i in range(len(csharp_files)):
        print("file", java_files[i])
        filter_files(java_files[i], csharp_files[i], "j2c-raw-data", "filtered_data")

In [ ]:
def make_tree_java(json, long_base_case=True):
    if general_base_cases(json) is not None:
        return general_base_cases(json)
    
    # hopefully just one
    for key in json.keys():
        
        # I could put angle brackets by why bother?
        tag = key.upper()
        if not long_base_case:
            if tag == "identifier" or tag == "value":
                return make_tree_csharp(json[key])
        else:
            parentNode = Node(tag)
            children = json[key]
            if type(children) is list:
                parentNode.children.extend(map(lambda child: 
                                       make_tree_csharp(child, long_base_case=long_base_case), 
                                       children))
            else:
                parentNode.children.append(make_tree_csharp(children, long_base_case=long_base_case))
    return parentNode 

def canonicalize_java(tree):
    var_names = {}
    value_names = {} # could be string, int, char, bool
    
    def make_generic(node, dict, symbol):
        if node.value in dict:
            node.value = dict[node.value]
        else:
            new_symbol = symbol + str(len(dict) + 1)
            dict[node.value] = new_symbol
            node.value = new_symbol

    
    def canonicalize_java_helper(tree):
        if tree.value == "IDENTIFIER":
            make_generic(tree.children[0], num_names, "a")
        elif tree.value == "VALUE":
            make_generic(tree.children[0], var_names, "v")
        else:
            for child in tree.children:
                canonicalize_java_helper(child)
                
    canonicalize_java_helper(tree)
    return tree


In [ ]:
def make_tree_csharp(json, long_base_case=True):
    print(json)
    
    if general_base_cases(json) is not None:
        return general_base_cases(json)
    
    # There should really only be one
    for key in json.keys():
        
        # I could put angle brackets by why bother?
        tag = key.upper()
        verbose_tokens = ["CharacterLiteralExpression", "NumericLiteralExpression", "StringLiteralExpression", "IdentifierName"]
        if not long_base_case and tag in verbose_tokens:
            if json[key] == {}:
                print("found one!")
                print(json)
            return make_tree_csharp(json[key])
        else:
            parentNode = Node(tag)
            children = json[key]
            if type(children) is list:
                parentNode.children.extend(map(lambda child: 
                                       make_tree_csharp(child, long_base_case=long_base_case), 
                                       children))
            else:
                parentNode.children.append(make_tree_csharp(children, long_base_case=long_base_case))
        print(">>", key)
    return parentNode 

# identifier
# value

def canonicalize_csharp(tree):
    num_names = {}
    var_names = {}
    char_names = {}
    string_names = {}
    
    def make_generic(node, dict, symbol):
        if node.value in dict:
            node.value = dict[node.value]
        else:
            new_symbol = symbol + str(len(dict) + 1)
            dict[node.value] = new_symbol
            node.value = new_symbol

    
    def canonicalize_csharp_helper(tree):
        if tree.value == "NUMERICLITERALEXPRESSION":
            make_generic(tree.children[0], num_names, "n")
        elif tree.value == "CHARACTERLITERALEXPRESSION":
            make_generic(tree.children[0], var_names, "c")
        elif tree.value == "STRINGLITERALEXPRESSION":
            make_generic(tree.children[0], string_names, "s")
        elif tree.value == "IDENTIFIERNAME":
            make_generic(tree.children[0], var_names, "v") #TODO: deal with argumentlist
        else:
            for child in tree.children:
                canonicalize_csharp_helper(child)
                
    canonicalize_csharp_helper(tree)
    return tree


def general_base_cases(json):
    # First base case - variable name
    if isinstance(json, string_types):
        return Node(make_var_name(json))

    # Second base case - variable value
    if type(json) is int:
        return Node(json)
    
    return None

def make_var_name(var_name):
    if var_name == 'h':
        return '<HEAD>'
    elif var_name == 't':
        return '<TAIL>'
    else:
        return var_name
    
    
    
    

In [ ]:
# csharp_json_lists = load_data("jsonified", "cs.txt")
java_json_lists = load_data("filtered_data", "java.txt")

In [ ]:
# # For the moment, ignore the itext dataset
# file_indices = [0, 1, 3, 4, 5] 

# for index in file_indices:
#     for i in range(5):
#         print("C#")
#         prog = csharp_json_lists[index][i]
#         c_tree = make_tree_csharp(prog)
#         better_c_tree = canonicalize_csharp(c_tree)
#         pretty_print_tree(better_c_tree)
#         print("JAVA")
#         prog = java_json_lists[index][i]
#         j_tree = make_tree_java(prog)
#         better_c_tree = canonicalize_csharp(j_tree)
#         pretty_print_tree(better_j_tree)
        


In [ ]:
def find_weirdness(tree):
    any_weird = False
    if type(tree) is dict:
        counter = 0
        for k in tree.keys():
            counter += 1
        if counter != 1:
            print(json.dumps(tree, indent=1))
            print("")
            return True
        if type(tree[k]) == str:
            pass
        elif type(tree[k]) == list:
            for child in tree[k]:
                any_weird = any_weird or find_weirdness(child)
        elif type(tree[k]) == dict:
            any_weird = find_weirdness(tree[k])        
    return any_weird

def find_weirdness2(tree):
    any_weird = False
    if type(tree) is dict:
        c_type = False
        for k in tree.keys():
            if c_type and type(tree[])
            counter += 1
        if counter != 1:
            print(json.dumps(tree, indent=1))
            print("")
            return True
        if type(tree[k]) == str:
            pass
        elif type(tree[k]) == list:
            for child in tree[k]:
                any_weird = any_weird or find_weirdness(child)
        elif type(tree[k]) == dict:
            any_weird = find_weirdness(tree[k])        
    return any_weird
    

found_cap = 20
i = 0
while found_cap > 0:
    prog = java_json_lists[1][i]
    if find_weirdness(prog):
        found_cap -= 1
    
    i+=1

In [ ]:
type("")